<a href="https://colab.research.google.com/github/CyberMaryVer/ProgressiveTransformersSLP/blob/master/text2gloss_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
! pip install wandb
import wandb

     |████████████████████████████████| 1.8MB 5.1MB/s 
     |████████████████████████████████| 143kB 41.4MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 143kB 39.9MB/s 
     |████████████████████████████████| 174kB 39.9MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=fb113feb815bffd2dde9f88fd1f8305ce850d5cf73451c8017f36f88dacdcc71
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=4447e4040205357d0be919e63bd87ad1dbf7518b00606643575c784f6bd66763
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll ha

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
%env WANDB_PROJECT=text2gloss

env: WANDB_PROJECT=text2gloss


In [4]:
! pip install --upgrade pip
! pip install datasets
! pip install transformers[sentencepiece]
! pip install git+https://github.com/huggingface/transformers.git

     |████████████████████████████████| 1.6MB 4.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 262 kB 6.5 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 243 kB 42.4 MB/s 
     |████████████████████████████████| 118 kB 67.7 MB/s 
     |████████████████████████████████| 127 kB 68.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.6
    Uninstalling urllib3-1.26.6:
      Successfully uninstalled urllib3-1.26.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wandb 0.11.0 requires urllib3>=1.26.5, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 2.5 MB 6.7 MB/s 
     |████████████████████████████████| 895 kB 44.7 MB/s 
     |████████████████████████████████| 3.3 MB 48.0 MB/s 
     |████████████████████████████████| 1.1 MB 69.0 MB/s 
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.14
    Uninstalling huggingface-hub-0.0.14:
      Successfully uninstalled huggingface-hub-0.0.14
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-6i4eyw07
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-6i4eyw07
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 636 kB 5.5 MB/s 
  Created wheel for transformers: filename=transformers-4.9.0.dev0-py3-none-any.whl size=2584040 sha256=09347af4bf5d0cd25c3bbc5b0c73b429ea2729c713f14dcb106303eadc0d26ba
  Stored in directory: /tmp/pi

# Fine-tuning a pretrained model

## Preparing the datasets

https://achrafothman.net/site/english-asl-gloss-parallel-corpus-2012-aslg-pc12/

In [ ]:
# aslg_pc12 # > 80 000 sentences

In [1]:
from datasets import load_dataset
dataset = load_dataset('aslg_pc12', split='train')

Using custom data configuration default


Dataset aslgpc12 downloaded and prepared to /root/.cache/huggingface/datasets/aslgpc12/default/0.0.1/7ae5d117644e44ff4d2233f27f1d5df93f429efab7b2ea84828516fcbce48fb4. Subsequent calls will reuse this data.


In [21]:
print(dataset.info.description)
print(dataset.info.features)
print(dataset.info.splits)

A large synthetic collection of parallel English and ASL-Gloss texts.
There are two string features: text, and gloss.

{'gloss': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}
{'train': SplitInfo(name='train', num_bytes=13475183, num_examples=87710, dataset_name='aslgpc12')}


In [14]:
assert len(dataset["gloss"]) == len(dataset["text"])

[custom datasets](https://huggingface.co/transformers/custom_datasets.html)

use custom dataset

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
# path_to_datasets = "/gdrive/MyDrive/glosses"
# ASLG = path_to_datasets + "/ASLG"
# SIGNUM = path_to_datasets + "/SIGNUM"

# ! ls $path_to_datasets
# ! ls $ASLG
# ! ls $SIGNUM

In [7]:
# def read_data_from_file(file):
#     with open(file, "r") as reader:
#         data = reader.readlines()
#         data = [s.replace("\n", "").lower() for s in data]
#         data = [s.replace(" x-", " X-").replace(" desc-", " DESC ") for s in data]
#     return data

In [ ]:
# test_file = ASLG + "/aslg.dev.gloss.asl"
# test = read_data_from_file(test_file)
# print(test)

## Preparing the model

In [10]:
# from transformers import T5ForConditionalGeneration, AutoTokenizer

# model = T5ForConditionalGeneration.from_pretrained('google/byt5-small')
# tokenizer = AutoTokenizer.from_pretrained('google/byt5-small')

In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-ase")
# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fr-ase")

In [12]:
# text_file_1 = ASLG + "/aslg.dev.en"
# label_file_1 = ASLG + "/aslg.dev.gloss.asl"
# text_file_2 = SIGNUM + "/signum.en"
# label_file_2 = SIGNUM + "/signum.gloss.asl"

# sentences = read_data_from_file(text_file_1) + read_data_from_file(text_file_2)
# glosses = read_data_from_file(label_file_1) + read_data_from_file(label_file_2)

In [16]:
sentences, glosses = dataset["text"], dataset["gloss"] #########################################

In [17]:
len(sentences), len(glosses)

(87710, 87710)

In [18]:
from sklearn.model_selection import train_test_split

# texts = tokenizer(sentences, padding="longest", return_tensors="pt")
# labels = tokenizer(glosses, padding="longest", return_tensors="pt").input_ids

train_texts, test_texts, train_labels, test_labels = train_test_split(sentences, glosses, test_size=.2)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1)

In [22]:
# special_tokens = {'additional_special_tokens': ['<X>', '<DESC>']}
# tokenizer.add_special_tokens(special_tokens)

train_encodings = tokenizer(train_texts, padding="longest", return_tensors="pt")
val_encodings = tokenizer(val_texts, padding="longest", return_tensors="pt")
test_encodings = tokenizer(test_texts, padding="longest", return_tensors="pt")

train_enc_labels = tokenizer(train_labels, padding="longest", return_tensors="pt").input_ids
val_enc_labels = tokenizer(val_labels, padding="longest", return_tensors="pt").input_ids
test_enc_labels = tokenizer(test_labels, padding="longest", return_tensors="pt").input_ids

In [ ]:
# loss = model(**model_inputs, labels=labels).loss

## Custom dataloader

In [23]:
import torch

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

    # def _build(self):
    #     for example in self.data:
    #         input_ = example[0]
    #         target = example[1]

    #         input_ = "paraphrase: " + input_ + ' </s>'
    #         target = target + " </s>"

    #         # tokenize inputs
    #         tokenized_inputs = self.tokenizer.batch_encode_plus(
    #             [input_], max_length=self.max_len, pad_to_max_length=True, truncation=True, return_tensors="pt"
    #         )
    #         # tokenize targets
    #         tokenized_targets = self.tokenizer.batch_encode_plus(
    #             [target], max_length=self.max_len, pad_to_max_length=True, truncation=True, return_tensors="pt"
    #         )

    #         self.inputs.append(tokenized_inputs)
    #         self.targets.append(tokenized_targets)

train_dataset = MyDataset(train_encodings, train_enc_labels)
val_dataset = MyDataset(val_encodings, val_enc_labels)
test_dataset = MyDataset(test_encodings, test_enc_labels)

## Trainer settings

In [25]:
path_to_datasets = "/content/results"

In [26]:
path_to_model = path_to_datasets + '/model1'
path_to_logs = path_to_datasets + '/logs1'

In [ ]:
# trainer.args

In [27]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=path_to_model,          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=path_to_logs,            # directory for storing logs
    logging_steps=10,
    tpu_num_cores=4,
    report_to="wandb",
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

# trainer.train()

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 63151
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15788
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: vsllabs (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Step,Training Loss
10,14.143900
20,13.531600
30,12.171600
40,10.058800
50,7.372800
60,4.965400
70,2.716700
80,1.542300
90,0.963300
100,0.667800


In [20]:
trainer.save_model(path_to_model)

Saving model checkpoint to /gdrive/MyDrive/glosses/model1
Configuration saved in /gdrive/MyDrive/glosses/model1/config.json
Model weights saved in /gdrive/MyDrive/glosses/model1/pytorch_model.bin


In [21]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 383
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


{'epoch': 2.0,
 'eval_loss': 0.1324140578508377,
 'eval_runtime': 173.0191,
 'eval_samples_per_second': 2.214,
 'eval_steps_per_second': 0.069}

# Example of use

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('./results/')

# from checkpoint
# config = AutoModelForSeq2SeqLMConfig.from_json_file('./results/my_tf_model_config.json')
# model = AutoModelForSeq2SeqLM.from_pretrained('./results/my_tf_checkpoint.ckpt.index', from_tf=True, config=config)

In [ ]:
def translate_text2gloss(input, model, tokenizer):
    output_len = len(input)
    input_ids = tokenizer(input, return_tensors="pt").input_ids  # Batch size 1
    outputs = model.generate(input_ids, max_length=output_len*2, min_length=output_len//2)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).upper()

    return decoded

input = "The weather is wonderful but rainy"
translate_text2gloss(input, model, tokenizer)

'X-WEAR BE WONDERFOL BUT RAINY BE <DESC> WONDERFOL BUT RAYNY <XESC> '

In [ ]:
num = 89

test_files = SIGNUM + "/signum.en"
test_labels = SIGNUM + "/signum.gloss.asl"
test_texts = read_data_from_file(test_files)
test_glosses = read_data_from_file(test_labels)
print(f'text: {test_texts[num]}\ngloss: {test_glosses[num]}')

input = test_texts[num]
translate_text2gloss(input, model, tokenizer)

text: the guided tour through the city centre is interesting.
gloss: city+centre <X> left guided-tour interesting 


'X-I GUIDE TOUR THROUGH CITY CENTRE CENTRE INTERESTING.'

In [ ]:
test_texts[90]

'on tuesdays the guided tour is expensive.'

[documentation](https://huggingface.co/docs/datasets/)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
model.save_pretrained(path_to_model+"/experiment3")

Configuration saved in /gdrive/MyDrive/glosses/model3/experiment3/config.json
Model weights saved in /gdrive/MyDrive/glosses/model3/experiment3/pytorch_model.bin


# For information

## Fine-tuning in PyTorch with the Trainer API

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

This will issue a warning about some of the pretrained weights not being used and some weights being randomly
initialized. That's because we are throwing away the pretraining head of the BERT model to replace it with a
classification head which is randomly initialized. We will fine-tune this model on our task, transferring the knowledge
of the pretrained model to it (which is why doing this is called transfer learning).

Then, to define our `Trainer`, we will need to instantiate a
`TrainingArguments`. This class contains all the hyperparameters we can tune for the
`Trainer` or the flags to activate the different training options it supports. Let's begin by
using all the defaults, the only thing we then have to provide is a directory in which the checkpoints will be saved:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

Then we can instantiate a `Trainer` like this:

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

To fine-tune our model, we just need to call

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=375, training_loss=0.30600254313151043, metrics={'train_runtime': 578.0235, 'train_samples_per_second': 5.19, 'train_steps_per_second': 0.649, 'total_flos': 789333166080000.0, 'train_loss': 0.30600254313151043, 'epoch': 3.0})

which will start a training that you can follow with a progress bar, which should take a couple of minutes to complete
(as long as you hav access to a GPU). It won't actually tell you anything useful about how well (or badly) your model
is performing however as by default, there is no evaluation during training, and we didn't tell the
`Trainer` to compute any metrics. Let's have a look on how to do that now!

To have the `Trainer` compute and report metrics, we need to give it a `compute_metrics`
function that takes predictions and labels (grouped in a namedtuple called `EvalPrediction`) and
return a dictionary with string items (the metric names) and float values (the metric values).

The 🤗 Datasets library provides an easy way to get the common metrics used in NLP with the `load_metric` function.
here we simply use accuracy. Then we define the `compute_metrics` function that just convert logits to predictions
(remember that all 🤗 Transformers models return the logits) and feed them to `compute` method of this metric.

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

The compute function needs to receive a tuple (with logits and labels) and has to return a dictionary with string keys
(the name of the metric) and float values. It will be called at the end of each evaluation phase on the whole arrays of
predictions/labels.

To check if this works on practice, let's create a new `Trainer` with our fine-tuned model:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_accuracy': 0.885,
 'eval_loss': 0.5189856886863708,
 'eval_runtime': 70.1239,
 'eval_samples_per_second': 14.26,
 'eval_steps_per_second': 1.783}

which showed an accuracy of 87.5% in our case.

If you want to fine-tune your model and regularly report the evaluation metrics (for instance at the end of each
epoch), here is how you should define your training arguments:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


See the documentation of `TrainingArguments` for more options.

<a id='keras'></a>

## Fine-tuning with Keras

Models can also be trained natively in TensorFlow using the Keras API. First, let's define our model:

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Then we will need to convert our datasets from before in standard `tf.data.Dataset`. Since we have fixed shapes,
it can easily be done like this. First we remove the *"text"* column from our datasets and set them in TensorFlow
format:

In [ ]:
tf_train_dataset = small_train_dataset.remove_columns(["text"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["text"]).with_format("tensorflow")

Then we convert everything in big tensors and use the `tf.data.Dataset.from_tensor_slices` method:

In [ ]:
train_features = {x: tf_train_dataset[x].to_tensor() for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_eval_dataset[x].to_tensor() for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

With this done, the model can then be compiled and trained as any Keras model:

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3)

With the tight interoperability between TensorFlow and PyTorch models, you can even save the model and then reload it
as a PyTorch model (or vice-versa):

In [ ]:
from transformers import AutoModelForSequenceClassification

model.save_pretrained("my_imdb_model")
pytorch_model = AutoModelForSequenceClassification.from_pretrained("my_imdb_model", from_tf=True)

<a id='pytorch_native'></a>

## Fine-tuning in native PyTorch

You might need to restart your notebook at this stage to free some memory, or excute the following code:

In [ ]:
del model
del pytorch_model
del trainer
torch.cuda.empty_cache()

Let's now see how to achieve the same results as in [trainer section](#trainer) in PyTorch. First we need to
define the dataloaders, which we will use to iterate over batches. We just need to apply a bit of post-processing to
our `tokenized_datasets` before doing that to:

- remove the columns corresponding to values the model does not expect (here the `"text"` column)
- rename the column `"label"` to `"labels"` (because the model expect the argument to be named `labels`)
- set the format of the datasets so they return PyTorch Tensors instead of lists.

Our *tokenized_datasets* has one method for each of those steps:

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Now that this is done, we can easily define our dataloaders:

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

Next, we define our model:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

We are almost ready to write our training loop, the only two things are missing are an optimizer and a learning rate
scheduler. The default optimizer used by the `Trainer` is `AdamW`:

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

Finally, the learning rate scheduler used by default it just a linear decay form the maximum value (5e-5 here) to 0:

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

One last thing, we will want to use the GPU if we have access to one (otherwise training might take several hours
instead of a couple of minutes). To do this, we define a `device` we will put our model and our batches on.

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

We now are ready to train! To get some sense of when it will be finished, we add a progress bar over our number of
training steps, using the *tqdm* library.

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Note that if you are used to freezing the body of your pretrained model (like in computer vision) the above may seem a
bit strange, as we are directly fine-tuning the whole model without taking any precaution. It actually works better
this way for Transformers model (so this is not an oversight on our side). If you're not familiar with what "freezing
the body" of the model means, forget you read this paragraph.

Now to check the results, we need to write the evaluation loop. Like in the [trainer section](#trainer) we will
use a metric from the datasets library. Here we accumulate the predictions at each batch before computing the final
result when the loop is finished.

In [ ]:
metric= load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

<a id='additional-resources'></a>

## Additional resources

To look at more fine-tuning examples you can refer to:

- [🤗 Transformers Examples](https://github.com/huggingface/transformers/tree/master/examples) which includes scripts
  to train on all common NLP tasks in PyTorch and TensorFlow.

- [🤗 Transformers Notebooks](https://huggingface.co/transformers/notebooks.html) which contains various notebooks and in particular one per task (look
  for the *how to finetune a model on xxx*).